[useful link to understand CLV](https://www.shopify.com/ie/blog/what-is-customer-lifetime-value#:~:text=Customer%20lifetime%20value%20(CLV%20or,their%20relationship%20with%20the%20company.)

In [1]:
import clickhouse_connect
import pandas as pd

In [2]:
client = clickhouse_connect.get_client(host='localhost',port="18123")

In [121]:
response=client.command('show tables')
print(response)

baskets
customers
products


In [120]:
client.command('drop table sales_mv')

In [126]:
query="""
CREATE TABLE sales
(
    `customer_id` UInt32,
    `basket_date` Date,
    `sales` AggregateFunction(sum, Decimal(9, 2))
)
ENGINE = AggregatingMergeTree
ORDER BY (customer_id, basket_date)
"""

In [127]:
join="""CREATE MATERIALIZED VIEW sales_mv
TO sales
AS
SELECT 
    customer_id,
    b.basket_date AS basket_date,
    sumState( taxful_price * basket_count ) AS sales   
FROM 
    baskets as b Join products as p 
    on b.product_id=p.product_id
GROUP BY
    customer_id,
    basket_date

"""

In [128]:
populating="""INSERT INTO sales
SELECT 
    customer_id,
    b.basket_date AS basket_date,
    sumState( taxful_price * basket_count ) AS sales   
FROM 
    baskets as b Join products as p 
    on b.product_id=p.product_id
GROUP BY
    customer_id,
    basket_date
"""

In [129]:
response=client.command(query)
print(response)

In [130]:
response=client.command(join)
print(response)

In [131]:
response=client.command(populating)
print(response)

In [35]:
df=client.query_df('SELECT * from products limit 2')
df.head()

,base_price,discount_percentage,quantity,manufacturer,tax_amount,product_id,category,sku,taxless_price,unit_discount_amount,min_price,discount_amount,created_on,product_name,price,taxful_price,base_unit_price
0,79.99,20.00,1,nan,0.00,49390,Men's Clothing,ZO0020300203,33.99,0.000,18.01,12.998,2016-12-10 12:50:24,Jersey dress - peacoat,15.00,24.99,154.99
1,28.99,0.00,1,Tigress Enterprises,0.00,52798,Women's Clothing,ZO0072100721,28.99,0.000,14.49,0.000,2016-12-12 10:37:55,Jumper - peacoat/winter white,28.99,28.99,28.99


In [34]:
df=client.query_df('SELECT * from customers limit 2')
df.head()

,customer_id,sex,customer_age,tenure
0,2093,male,47,133
1,12817,male,40,132


In [183]:
df=client.query_df('SELECT * from baskets as b Join products as p on b.product_id=p.product_id' )[['product_id','customer_id','basket_date','basket_count','taxful_price']]
df.head()

,product_id,customer_id,basket_date,basket_count,taxful_price
0,6896403,4784,2019-05-27,2,28.99
1,3328663,8314,2019-05-25,2,43.99
2,3766775,8314,2019-05-25,3,13.99
3,2268129,8857,2019-05-25,2,50.00
4,46741621,9273,2019-06-07,2,54.99


In [184]:
df['sales']=df['basket_count']*df['taxful_price']

In [185]:
grouped=df.groupby(['customer_id'])['sales'].sum()

In [186]:
gp=grouped.to_frame()
gp.head()

,sales
customer_id,
4784,57.98
8314,129.95
8857,100.00
9273,109.98
11172,169.98


In [187]:
gp.loc[(17170040,)]

sales    21.98
Name: 17170040, dtype: object

In [182]:
df=client.query_df('select customer_id,basket_date,sumMerge(sales) as sales from sales_mv GROUP BY customer_id,basket_date')
df.head()

,customer_id,basket_date,sales
0,17170040,2019-05-26,21.98
1,43762876,2019-06-10,125.97
2,33273585,2019-06-03,27.98
3,12173890,2019-06-15,54.95
4,6530798,2019-05-22,119.98


In [134]:
df[df["customer_id"]==17170040]

,customer_id,basket_date,sales
0,17170040,2019-05-26,21.98


In [196]:
grouped_by_customer=df.groupby(['customer_id'])['sales'].sum().to_frame()

In [197]:
grouped_by_customer

,sales
customer_id,
4784,57.98
8314,129.95
8857,100.00
9273,109.98
11172,169.98
...,...
44460516,57.98
44461180,31.98
44473609,49.98
